# Traitement des boursiers sanitaires et sociaux de BFC (Bourgogne Franche Comte)


## Procedure
1. Chargement du fichier boursiers sanitaires et sociaux BFC (Bourgogne Franche Comte)
2. Nettoyage des données et premier mapping au bon format de données attendu dans la BDD
3. Application des critères sur les données du CNOUS
4. Cleanup (date de naissance + 4 heures)
5. Ajout des valeurs pour les colonnes par défault (Paris 13 pour commune naissance et code insee naissance)
6. Output to CSV

In [ ]:
import os

from dotenv import load_dotenv
import pandas as pd
import json
from datetime import datetime
import numpy as np

load_dotenv()

cnous_filepath = os.environ['CNOUS_PATHFILE']

base_output_filepath = os.environ['DB_EXPORT']

defaults = {
  'code_organisme' : '2700',
  'code_insee_naissance' : '75113',
  'commune_naissance' : 'PARIS',
  'code_iso_pays_naissance' : 'FR',
  'pays_naissance' : 'FRANCE',
  'organisme' : 'cnous',
  'situation' : 'boursier'
}

In [ ]:
cnous_df = pd.read_csv(cnous_filepath, encoding='utf-8', on_bad_lines='skip', sep=';', engine="c",dtype=str)

In [ ]:
# map CNOUS
cnous_column_mapping = {
  'a-matricule': 'allocataire-matricule',
   # M or MME
  'a-qualite': 'allocataire-qualite',
  'nom': 'allocataire-nom',
  'prenom': 'allocataire-prenom',
  'date_naissance': 'allocataire-date_naissance',
  'a-courriel': 'allocataire-courriel',
  'a-telephone': 'allocataire-telephone',

  # adresse allocataire
  'a-a-voie': 'adresse_allocataire-voie',
  'a-a-code_postal': 'adresse_allocataire-code_postal',
  'a-a-commune': 'adresse_allocataire-commune',

  # Add leading 0, some have only 4 digits ...
  'a-a-code_insee': 'adresse_allocataire-code_insee',
  'a-a-cplt_adresse': 'adresse_allocataire-cplt_adresse',
  'a-a-nom_adresse_postale': 'adresse_allocataire-nom_adresse_postale',

  # infos bénéficiaires
  # M for M,
  # MME for F
  'genre': 'genre',
}

# Drop unused column
df_psp_mapped_cnous = cnous_df.copy()

df_psp_mapped_cnous.drop(columns=[
    'a-code_insee_commune_naissance', 'a-commune_naissance', 'a-code_iso_pays_naissance', 'a-nom', 'a-prenom', 'a-date_naissance'
], inplace=True)

df_psp_mapped_cnous.rename(columns=cnous_column_mapping, inplace=True)

# organisme
df_psp_mapped_cnous['organisme'] = defaults['organisme']
df_psp_mapped_cnous['situation'] = defaults['situation']

# We hardcode those since the boursiers from BFC (Bourgogne Franche Comté) do not have these
# And in the form pass Sport, it will be automatically bypassed
df_psp_mapped_cnous['allocataire-code_iso_pays_naissance'] = defaults['code_iso_pays_naissance']
df_psp_mapped_cnous['allocataire-pays_naissance'] = defaults['pays_naissance']
df_psp_mapped_cnous['allocataire-code_insee_commune_naissance'] = defaults['code_insee_naissance']
df_psp_mapped_cnous['allocataire-commune_naissance'] = defaults['commune_naissance']

# Unique code organisme for this list of boursiers sanitaires et sociaux BFC
df_psp_mapped_cnous['allocataire-code_organisme'] = defaults['code_organisme']

df_psp_mapped_cnous['nom'] = df_psp_mapped_cnous['allocataire-nom']
df_psp_mapped_cnous['prenom'] = df_psp_mapped_cnous['allocataire-prenom']

df_psp_mapped_cnous['genre'] = df_psp_mapped_cnous['allocataire-qualite'].str.strip().replace('MME', 'F')
df_psp_mapped_cnous['allocataire-qualite'] = df_psp_mapped_cnous['allocataire-qualite'].str.strip().replace({'MME': 'Mme', 'M': 'M'})

In [ ]:
# Birth date
df_psp_mapped_cnous['date_naissance'] = pd.to_datetime(
    df_psp_mapped_cnous['allocataire-date_naissance'],
    format='%m/%d/%y'
)

df_psp_mapped_cnous['allocataire-date_naissance'] = df_psp_mapped_cnous['date_naissance'].dt.strftime('%m/%d/%y')

In [ ]:
# apply criterias on CNOUS datas
from datetime import timedelta
from dateutil.relativedelta import relativedelta

# Cut off date for eligibility for year 2024 
end_date = pd.to_datetime('2024-10-15').date()
start_date = end_date - relativedelta(years=28)

cnous_situation_mask = (df_psp_mapped_cnous['date_naissance'].dt.date >= start_date) & (
    df_psp_mapped_cnous['date_naissance'].dt.date <= end_date)

df_psp_mapped_cnous_filtered = df_psp_mapped_cnous[cnous_situation_mask]

print(f"{len(df_psp_mapped_cnous) - len(df_psp_mapped_cnous_filtered)} rows for CNOUS dataframe were removed based on criterias")

# Merge dans un seul dataframe cible pour BDD Postgresql

In [ ]:
# concat into a single dataframe
df_all = pd.concat([df_psp_mapped_cnous_filtered], axis=0, ignore_index=True)

# remove rows with missing necessary values (if one of those value are missing we cannot generate a code)
necessary_column = ['nom', 'prenom', 'date_naissance', 'genre']
df_all_valid_row = df_all.dropna(subset=necessary_column)

# remove columns with all null value
df_all_valid = df_all_valid_row.dropna(axis=1, how='all')

assert len(
  df_all_valid[df_all['nom'].isnull() | df_all_valid['prenom'].isnull() | df_all_valid['date_naissance'].isnull()]
) == 0

In [ ]:
import unicodedata

def unaccent_and_upper(text):
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ASCII', 'ignore').decode('utf-8')
    return text.upper()

In [ ]:
# Upper case these columns for the merge
df_all_valid['prenom'] = df_all_valid['prenom'].astype(str).apply(unaccent_and_upper)
df_all_valid['nom'] = df_all_valid['nom'].astype(str).apply(unaccent_and_upper)
df_all_valid['genre'] = df_all_valid['genre'].astype(str).apply(lambda x: x.upper())

In [ ]:
# lower case on emails on all
df_all_valid['allocataire-courriel'] = df_all_valid['allocataire-courriel'].str.lower()

In [ ]:
# remove rows when beneficiary is before september 1993
mask_before_1993 = pd.to_datetime(df_all_valid['date_naissance']) > datetime(1993, 9, 16)
df_all_valid_after93 = df_all_valid[mask_before_1993]

print(f"{len(df_all_valid) - len(df_all_valid_after93)} rows where removed because date_naissance was before 1993")

In [ ]:
# add 4h on all birthdates
df_all_valid_after93.loc[:, 'date_naissance'] = df_all_valid_after93['date_naissance'] + timedelta(hours=4)

In [ ]:
# remove duplicate beneficiaries
df_all_valid_no_duplicate = df_all_valid_after93.drop_duplicates(subset=[
  'date_naissance',
  'nom',
  'prenom',
  'genre',
  'organisme',
  'situation',
  'allocataire-qualite',
  'allocataire-matricule',
  'allocataire-prenom',
  'allocataire-date_naissance',
  'allocataire-courriel'
])

print(f"{len(df_all_valid_after93) - len(df_all_valid_no_duplicate)} duplicate rows were removed")

In [ ]:
# map to json values for target DB model 
## map allocataire json
def to_json_allocataire_without_null(row):
  allocataire_mapping = {
    'qualite': row['allocataire-qualite'],
    'matricule': row['allocataire-matricule'],
    'nom': row['allocataire-nom'],
    'prenom': row['allocataire-prenom'],
    'date_naissance': row['allocataire-date_naissance'],
    'courriel': row['allocataire-courriel'],
    'telephone': row['allocataire-telephone'],
    'code_insee_commune_naissance': row['allocataire-code_insee_commune_naissance'],
    'commune_naissance': row['allocataire-commune_naissance'],
    'code_iso_pays_naissance': row['allocataire-code_iso_pays_naissance'],
    'pays_naissance': row['allocataire-pays_naissance'],
    'code_organisme': row['allocataire-code_organisme']
  }
  filtered_NaN_allocataire = {k: v for k, v in allocataire_mapping.items() if pd.notnull(v)}
  return json.dumps(filtered_NaN_allocataire, ensure_ascii=False)


df_all_valid_no_duplicate['allocataire'] = df_all_valid_no_duplicate.apply(to_json_allocataire_without_null, axis=1)

In [ ]:
## map adresse_allocataire json
def to_json_adresse_without_null(row):
  adresse_mapping = {
    'voie': row['adresse_allocataire-voie'],
    'code_postal': format(pd.to_numeric(row['adresse_allocataire-code_postal'], errors='coerce'), '05d'),
    'commune': row['adresse_allocataire-commune'],
    'code_insee': format(pd.to_numeric(row['adresse_allocataire-code_insee'], errors='coerce'), '05d'),
    'cplt_adresse': str(row['adresse_allocataire-cplt_adresse']).replace("\"", ''),
    'nom_adresse_postale': row['adresse_allocataire-nom_adresse_postale'],
  }

  filtered_address = {k: v for k, v in adresse_mapping.items() if pd.notnull(v)}
  return json.dumps(filtered_address, ensure_ascii=False)


df_all_valid_no_duplicate['adresse_allocataire'] = df_all_valid_no_duplicate.apply(to_json_adresse_without_null, axis=1)

In [ ]:
## drop null value
df_final = df_all_valid_no_duplicate.drop(columns=[
  'allocataire-qualite',
  'allocataire-matricule',
  'allocataire-nom',
  'allocataire-prenom',
  'allocataire-date_naissance',
  'allocataire-courriel',
  'allocataire-telephone',
  'allocataire-code_insee_commune_naissance',
  'allocataire-commune_naissance',
  'allocataire-code_iso_pays_naissance',
  'allocataire-pays_naissance',
  'allocataire-code_organisme',
  'adresse_allocataire-voie',
  'adresse_allocataire-code_postal',
  'adresse_allocataire-commune',
  'adresse_allocataire-code_insee',
  'adresse_allocataire-cplt_adresse',
  'adresse_allocataire-nom_adresse_postale',
])


In [ ]:
import pytz
from datetime import datetime

tz = pytz.timezone('Europe/Paris')
now = datetime.now()
now_tz = tz.localize(now)

# Add missing default column needed for target DB model
df_final['id'] = np.NaN
df_final['exercice_id'] = 3
df_final['uuid_doc'] = np.NaN
df_final['id_psp'] = np.NaN
df_final[['zrr', 'qpv', 'a_valider', 'refuser']] = False
df_final[['created_at', 'updated_at']] = now_tz

In [ ]:
# load all existing codes
existing_codes_db_filepath = os.environ['DB_EXISTING_CODES']

df_existing_codes = pd.read_csv(existing_codes_db_filepath, encoding='utf-8', dtype=str)

In [ ]:
df_no_code = df_final[df_final['id_psp'].isna()]

In [ ]:
# generate new code ensuring no duplicates with existings
import random
import string
import datetime

current_date = datetime.datetime.now()
current_year = str(current_date.year)[-2:]

def get_characters_set(size = 4):
    return ''.join(random.choices([c for c in string.ascii_uppercase if c not in 'OI'], k=size))

def generate_code():
    return f"{current_year}-{get_characters_set(4)}-{get_characters_set(4)}"

# init set of codes with existing
unique_codes = set(df_existing_codes['id_psp'])

# init current_code count
current_codes_count = len(unique_codes)

while len(unique_codes) < current_codes_count + len(df_no_code):
    code = generate_code()
    unique_codes.add(code)

# only retrieve newly created codes
new_codes = list(unique_codes.difference(df_existing_codes['id_psp']))
df_new_codes = pd.DataFrame({ 'id_psp': new_codes })

print(f"{len(df_new_codes)} generated codes")

In [ ]:
df_no_code = df_no_code.reset_index(drop=True).combine_first(df_new_codes.reset_index(drop=True))

In [ ]:
df_no_code[[
    'id',
    'id_psp',
    'nom',
    'prenom',
    'date_naissance',
    'genre',
    'organisme',
    'situation',
    'allocataire',
    'adresse_allocataire',
    'created_at',
    'updated_at',
    'qpv',
    'a_valider',
    'exercice_id',
    'zrr',
    'uuid_doc',
    'refuser'
]].to_csv('./with-codes.csv', index=False)